In [1]:
import stanza
import spacy_stanza
# import dadmatools.pipeline.language as language
# from dadmatools.models.normalizer import Normalizer
from parsivar import Normalizer as ParsivarNormalizer
import json 
import pytse_client as tse
import warnings
from spacy.language import Language
from spacy.tokens import Span
import pandas as pd
from tqdm import tqdm 
import spacy
from spacy.matcher import Matcher, PhraseMatcher
from spacy.tokens import Span
from spacy import displacy

import re

2022-04-14 19:57:18.882919: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-14 19:57:18.882950: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Setup Stanza

In [2]:
stanza.install_corenlp()

2022-04-14 19:57:21 WARNING: Directory /home/ahura/stanza_corenlp already exists. Please install CoreNLP to a new directory.


In [3]:
stanza.download('fa')

2022-04-14 19:57:22 INFO: Downloading default packages for language: fa (Persian)...
2022-04-14 19:57:22 INFO: File exists: /home/ahura/stanza_resources/fa/default.zip.
2022-04-14 19:57:26 INFO: Finished downloading models and saved to /home/ahura/stanza_resources.


In [4]:
stanza_nlp = spacy_stanza.load_pipeline("fa")

2022-04-14 19:57:27 INFO: Loading these models for language: fa (Persian):
| Processor | Package |
-----------------------
| tokenize  | perdt   |
| mwt       | perdt   |
| pos       | perdt   |
| lemma     | perdt   |
| depparse  | perdt   |

2022-04-14 19:57:27 INFO: Use device: gpu
2022-04-14 19:57:27 INFO: Loading: tokenize
2022-04-14 19:57:28 INFO: Loading: mwt
2022-04-14 19:57:28 INFO: Loading: pos
2022-04-14 19:57:29 INFO: Loading: lemma
2022-04-14 19:57:29 INFO: Loading: depparse
2022-04-14 19:57:29 INFO: Done loading processors!


# Setup DadmaTools

In [5]:
# # here lemmatizer and pos tagger will be loaded
# # as tokenizer is the default tool, it will be loaded as well even without calling
# pips = 'ner, pos, dep, cons, chunk, lem, tok' 
# dadma_nlp = language.Pipeline(pips)

# # you can see the pipeline with this code
# print(dadma_nlp.analyze_pipes(pretty=True))

# # dadma_doc is an SpaCy object

### Dadma Normalizer

In [6]:
# normalizer1 = Normalizer(
#     full_cleaning=False,
#     unify_chars=True,
#     refine_punc_spacing=True,
#     remove_extra_space=True,
#     remove_puncs=False,
#     remove_html=False,
#     remove_stop_word=False,
#     replace_email_with="<EMAIL>",
#     replace_number_with=None,
#     replace_url_with="<URL",
#     replace_mobile_number_with="<MOBILE_NUMBER>",
#     replace_emoji_with="<EMOJI>",
#     replace_home_number_with="<HOME_NUMBER>"
# )

normalizer2 = ParsivarNormalizer()

# Test Libs

# Read & Write Symbols 

## Download And Write

In [7]:

# warnings.simplefilter(action='ignore', category=FutureWarning)

# symbol_to_price_records = tse.download(symbols="all", write_to_csv=True)

In [8]:
# symbols = list(symbol_to_price_records.keys())
# symbols = sorted(symbols, key=str.lower)


# with open('symbols.json', 'w',) as file:
# json.dump({"symbols": symbols}, file, ensure_ascii=False)
    


In [9]:


# columns = ["Symbol", "Corp. Title", "TSE URL", "Group Name"]

# symbols_info_list = []
# for symbol in tqdm(x.keys()):
#     ticker = tse.Ticker(symbol, adjust=True, )

#     row = [symbol, ticker.title, ticker.url, ticker.group_name]

#     symbols_info_list.append(row)

# pd.DataFrame(symbols_info_list, )


## Write

In [10]:
with open('symbols.json', 'r') as file:
    symbols = json.load(file)['symbols']
    
len(symbols)

1200

# Sample Text

In [39]:
texts = [
    "نماد برکت افزایش و نماد کگل کاهش یافت.",
    "برکت امروز اطلاعیه‌ای مهم منتشر میکند.",
    "نماد برکت امروز عرضه‌ی اولیه خیلی خوبی داره.",
    "نماد برکت امروز عرضه ی اولیه خیلی خوبی داره.",
    "نماد برکت امروز عرضه ی اولیه دارد.",
    "نماد برکت امروز عرضه ی اولیه خیلی خوبی داره.",
    "عرضه های اولیه امروز خوب هستند.",
    "عرضه اولیه‌های امروز خوب هستند.",
    "برکت همین افشای ب باعث می شود سهم سه درصد مثبت بشود. بخاطر همین میگم پیگیر باشید.",
    'روز چهارشنبه یک دفعه برای خودشون افشا زدن.',
    "سهام وغدیر و خزر کاهش یافت.",
    "برای خودشون ی افشایی زدن.",
    "آ س پ امروز بالا رفت.",
    "این سهم تا کنون سودهای زیادی داده است.",
    "این سهم تا کنون سود های زیادی داده است.",
    # "ریزش بازار به دلیل حمله‌ی روسیه هست.",
    # "فک کنم یه اصلاح قیمتی و کمی ریزش داشته باشیم.",
    # "قرارداد با آمریکا باعث افت قیمت سهم وغدیر شد",
    # "یک نکته‌ی تکنیکالی هم در صورت دستکاری نشدن اضافه کنم، کندلی که روز سه شنبه‌ی گذشته ثبت کرد کامل است",
    # "روز چهارشنبه یه دفعه برای خودشون افشا زدن",
    # "رشد قیمت‌ها باعث ایجاد صف خرید در سهم پرشیا شد",
    # "شاخص به ۲ میلیون می‌رسه",
    # "آمریکا باعث ریزش بازار شد",
    # "آمریکا موجب ریزش بازار شد",
    # "آمریکا دلیل ریزش بازار شد",
    # "کاهش قیمت سهم عجیب بود",
    # "قیمت زیاد شد",
    # "قیمت زیاد است",
    # "به کتابخانه رفتم.",
    # "به کتابخانه رفت.",
    # "پول در جیب من است.",
    # "سهم قیمتش پایین است",
    # "حضور تو موجب خوشحالی من در هوای بارانی است",
]

stock_terms = [
    'ضرر',
    'سود',
    'اطلاعیه',
    'افزایش سرمایه',
    'تقسیم سود',
    'دامنه نوسان',
    'نوسان شدید',
    'سهم رانتی',
    'عرضه اولیه',
    'افشا',
    'فعالیت ماهانه',
    'فعالیت سالانه',
    'کاهش',
    'افزایش',
    'بالا',
    'پایین'
]

In [40]:
matcher: Matcher = Matcher(stanza_nlp.vocab, validate=True)

patterns = []
for term in stock_terms:
    term_tokens = stanza_nlp.tokenizer(term)
    # y = r'([\u200c|ی|ا|]{0,4})|(ی\s)'
    # y = r'[\u200c|ی|ا]{,4}'
    
    pre_det = ["این", "هر", "چند", "چندین", "آن", "همین", "همان", "چنین", "چنین"]
    
    if len(term_tokens) == 1:
        tok1_text = term_tokens[0].text
        pattern = [
            {'TEXT': {"IN": pre_det}, "OP": "?"},
            {'TEXT': {"REGEX": tok1_text}},
        ]

    elif len(term_tokens) == 2:
        tok1_text = term_tokens[0].text
        tok2_text = term_tokens[1].text
        pattern = [
            {'TEXT': {"IN": pre_det}, "OP": "?"},
            {'TEXT': {"REGEX": tok1_text}},
            {'TEXT': {"IN": ["ای", "ی", "ها", "های"]}, "OP": "?"},
            {'TEXT': {"IN": pre_det}, "OP": "?"},
            {'TEXT': {"REGEX": tok2_text}},
        ]
        
    patterns.append(pattern)

matcher.add("TERM", patterns, greedy="LONGEST")

# for text in texts:
#     doc = stanza_nlp(text)
#     print("Text: " + text)
#     # for id, start, end in matcher(doc):
#     #     print(stanza_nlp.vocab.strings[id], doc[start:end])

#     for id, start, end in matcher(doc): 
#         print(id, start, end)
#         m = doc[start: end]
#         print(m)
    
    # term_spans = list(map(lambda match: doc.char_span(match[1], match[2], label="TERM"),
#                         #   matcher(doc)))
    
#     # print(term_spans)

#     print("\n")


In [42]:
def convert_subtree_to_str(token, not_in=[], remove_ADP=False):
    extracted_event = ''
    start = None
    end = None
    for x in token.subtree:
        if x.pos_ in not_in:
            continue
        # print('1', x, x.pos_, x.head)
        if remove_ADP and x.pos_ == 'ADP':
            if x.head.text == token.text:
                continue

        extracted_event += str(x) + ' '
        if start is None:
            start = x.idx
        end = x.idx + len(x.text)
    extracted_event = extracted_event[:-1]
    # print('--- in convert s2s ', extracted_event)
    return extracted_event, start, end


def create_output(output_type, marker, span, **kwargs):
    defaults = {
        "type":output_type, 
        "marker":marker, 
        "span":repr(span),
        }
    return {
        **defaults,
        **kwargs
    }


import re
from spacy.util import filter_spans



TERM = "TERM"
SYMBOL = "SYMBOL"

for text_index, text in enumerate(texts):
    # text = normalizer1.normalize(text)
    text = normalizer2.normalize(text)

    print(f'Text {text_index + 1}: ...')
    print(f"Normalized Text: {text}")

    doc = stanza_nlp(text)

    expression = "|".join(symbols)
    
    symbol_spans = list(map(lambda match: doc.char_span(*match.span(), label=SYMBOL),
                            re.finditer(expression, text)))

    symbol_spans = list(filter(lambda span: span is not None,
                               symbol_spans))

    term_spans = list(map(lambda match: Span(doc, match[1], match[2], label=TERM),
                          matcher(doc)))
    
    term_spans = list(filter(lambda span: span is not None,
                             term_spans))
        
    spans = symbol_spans + term_spans
    
    print(term_spans)
    
    doc.set_ents(spans)
    
    with doc.retokenize() as retokenizer:
        attrs = {'POS': "NOUN"}
        for span in symbol_spans:
            retokenizer.merge(span, attrs)

    with doc.retokenize() as retokenizer:
        attrs = {'POS': "NOUN"}
        for span in term_spans:
            retokenizer.merge(span, attrs)

    # print('Entities: ...')
    # displacy.render(doc, style='ent')
    
    # print('Tokens: ...')
    # print(list(doc[:]))
    
    print('Dependency Tree: ...')
    displacy.render(doc, style='dep')
    
    term_ents = list(filter(lambda ent: ent.label_ == TERM,
                     doc.ents))
    
    from spacy.symbols import nsubj, VERB, NOUN
    
    for term_ent in term_ents:
        token = term_ent[0]
        
        if token.dep_ == 'compound:lvc':
            print('type is: coumpound:lvd')
            print('parent is: ', token.head)
            if token.head.pos == VERB:
                print('parent was a verb')
                print('children of parent: ', list(token.head.children))
                subject = None
                for child in token.head.children:
                    if child.dep == nsubj:
                        print('Found a subject, outputing subtree of subject as subject')
                        subject, start_subj, end_subj = convert_subtree_to_str(child)
                        break
                    else:
                        print('NOT IMP - child dep is: ', child.dep_)
                        pass
            # else:e
            # print('didn\'t found any subj')

                start = token.idx
                end = token.idx + len(token.text) + 1 + len(token.head.text)
                extracted_event = token.text + ' ' + token.head.text #TODO: check
                if subject:
                    print(create_output('واقعه', extracted_event, (start,end), subject=subject))
                else:
                    print(create_output('واقعه', extracted_event, (start,end)))

            # elif token.dep_ == 'nmod':
            #     print('type is: nmod')
            #     print('parent is: ', token.head)

            #     print('finding parent that is not nmod')
            #     main_noun = token
            #     while main_noun.dep_ == 'nmod' and main_noun.head.pos == NOUN:
            #       main_noun = main_noun.head
            #     if main_noun.pos != NOUN:
            #       continue

            #     print('Found a Noun parent that is not nmod')
            #     extracted_event, start, end = convert_subtree_to_str(token.head, not_in=['ADP'])

        elif token.dep_ == 'nmod' or token.dep_ == 'amod':
            print('type is: ', token.dep_)
            print('parent is: ', token.head)

            print('finding parent that is not nmod or amod')
            main_noun = token
            # print('1', main_noun, main_noun.dep_, main_noun.head.pos_)
            while (main_noun.dep_ == 'nmod' or main_noun.dep_ == 'amod') and main_noun.head.pos == NOUN:
                main_noun = main_noun.head
                # print('2', main_noun, main_noun.dep_, main_noun.head.pos_)
            if main_noun.pos != NOUN:
                continue

            print('Found a Noun parent that is not nmod or amod')
            extracted_event, start, end = convert_subtree_to_str(main_noun, remove_ADP=True)
            # extracted_event, start, end = convert_subtree_to_str(main_noun, not_in=['ADP'])

            print(create_output('واقعه', extracted_event, (start,end)))

        else:
            print('type is: other')
            extracted_event, start, end = convert_subtree_to_str(token, remove_ADP=True)

            # print(text[start:end])
            print(create_output('واقعه', extracted_event, (start,end)))

           
    
    print('-' * 75)

# stock_terms.extend(extended_stock_terms)

# for text in texts:
    # doc = stanza_nlp(text)
    # print("Text: " + text)
    # for id, start, end in matcher(doc):
    #     print(stanza_nlp.vocab.strings[id], doc[start:end])

    # pattern = "|".join(stock_terms)

    # print("Text: " + text)
    # for match in re.finditer(pattern, text):
        # s, e = match.span()
        # print(f"Match Found: ({text[s:e]})")

    # print("\n")

# stock_terms


[W109] Unable to save user hooks while serializing the doc. Re-add any required user hooks to the doc after processing.


Text 1: ...
Normalized Text: نماد برکت افزایش و نماد کگل کاهش یافت .
[افزایش, کاهش]
Dependency Tree: ...


[W109] Unable to save user hooks while serializing the doc. Re-add any required user hooks to the doc after processing.


type is:  nmod
parent is:  برکت
finding parent that is not nmod or amod
Found a Noun parent that is not nmod or amod
{'type': 'واقعه', 'marker': 'نماد برکت افزایش و نماد کگل', 'span': '(0, 27)'}
type is: coumpound:lvd
parent is:  یافت
parent was a verb
children of parent:  [نماد, کاهش, .]
Found a subject, outputing subtree of subject as subject
{'type': 'واقعه', 'marker': 'کاهش یافت', 'span': '(28, 37)', 'subject': 'نماد برکت افزایش و نماد کگل'}
---------------------------------------------------------------------------
Text 2: ...
Normalized Text: برکت امروز اطلاعیه‌ای مهم منتشر می‌کند .
[اطلاعیه‌ای]
Dependency Tree: ...


[W109] Unable to save user hooks while serializing the doc. Re-add any required user hooks to the doc after processing.


...

[سودهای]
Dependency Tree: ...


type is: other
{'type': 'واقعه', 'marker': 'سودهای زیادی', 'span': '(16, 28)'}
---------------------------------------------------------------------------
Text 15: ...
Normalized Text: این سهم تا کنون سود‌های زیادی داده است .


[W109] Unable to save user hooks while serializing the doc. Re-add any required user hooks to the doc after processing.


[سود‌های]
Dependency Tree: ...


type is: other
{'type': 'واقعه', 'marker': 'سود\u200cهای زیادی', 'span': '(16, 29)'}
---------------------------------------------------------------------------


# Symbol Tagging Component

# Stanza

## POS Matching 

In [14]:
# for t_index, text in enumerate(texts): 
#     text = normalizer1.normalize(text)
#     text = normalizer2.normalize(text)
    
#     print(f"Normalized Text: {text}")
#     doc = stanza_nlp(text)
#     print(f'Text {t_index + 1}: ...')
    
#     # for m_index, (match_id, start, end) in enumerate(matches):
#     #     span:Span = Span(doc, start, end, label=match_id)
#     #     print(f"Match {m_index + 1}: {span.text}, {span.label_}")
#     #     if span.label_ == "term_pattern":
#     #         print(list(span.subtree))
        
#     # for s_index, sentence in enumerate(doc.sents):
#     #     print(f'Sentence {s_index + 1}: ...')

#     print(doc.ents)

#     displacy.render(doc, style="ent")
#     print("\n\n")

## Dependency Tree

In [15]:
# def children_string(children): 
#     return " ".join([ch.text for ch in children])
    
# # stanza_nlp.add_pipe("merge_noun_chunks")



# for t_index, text in enumerate(texts):
#     text = normalizer.normalize(text)
#     doc = stanza_nlp(text)
#     print(f'Text {t_index + 1}: ...')
        
#     for s_index, sentence in enumerate(doc.sents):
#         print(f'Sentence {s_index + 1}: ...')
#         # for token in sentence:
#             # print(f'word: {token.text:12}, pos: {token.pos_:10}, tag: {token.tag_:10}, dep: {token.dep_:15}')

#         print(f'\n\n')
#         print(f"sentence root: {sentence.root.text}")
#         for child1 in sentence.root.children:
#             print(f"{child1.text}, {child1.pos_}, {child1.tag_} {child1.dep_}")
            
            
        
#         displacy.render(sentence, style="dep")
        
                
#     print("\n\n")


# Dadma

In [16]:

# def children_string(children): 
#     return " ".join([ch.text for ch in children])

    
# for t_index, text in enumerate(texts):
#     text = normalizer.normalize(text)
#     doc = dadma_nlp(text)
#     print(f'Text {t_index + 1}: ...')
        
#     for s_index, sentence in enumerate(doc.sents):
#         print(f'Sentence {s_index + 1}: ...')
#         # for token in sentence:
#         #     print(f'word: {token.text:12}, pos: {token.pos_:10}, tag: {token.tag_:10}, dep: {token.dep_:15}')

#         print(f'\n\n')
#         print(f"sentence root: {sentence.root.text}")
#         # for child1 in sentence.root.children:
#             # print(f"{child1.text} Span is: {children_string(child1.subtree)}")
            
#         # print(f"constituency: {doc._.constituency}")
#         print(f"chunks: {doc._.chunks}")
        
    
                
#     print("\n\n")


# TestTest

## display Matchings

In [17]:


# nlp = spacy.load("en_core_web_sm")
# matcher = Matcher(nlp.vocab)
# matched_sents = []  # Collect data of matched sentences to be visualized

# def collect_sents(matcher, doc, i, matches):
#     match_id, start, end = matches[i]
#     span = doc[start:end]  # Matched span
#     sent = span.sent  # Sentence containing matched span
#     # Append mock entity for match in displaCy style to matched_sents
#     # get the match span by ofsetting the start and end of the span with the
#     # start and end of the sentence in the doc
#     match_ents = [{
#         "start": span.start_char - sent.start_char,
#         "end": span.end_char - sent.start_char,
#         "label": "MATCH",
#     }]
#     matched_sents.append({"text": sent.text, "ents": match_ents})

# pattern = [{"LOWER": "facebook"}, {"LEMMA": "be"}, {"POS": "ADV", "OP": "*"},
#            {"POS": "ADJ"}]
# matcher.add("FacebookIs", [pattern], on_match=collect_sents)  # add pattern
# doc = nlp("I'd say that Facebook is evil. – Facebook is pretty cool, right?")
# matches = matcher(doc)

# # Serve visualization of sentences containing match with displaCy
# # set manual=True to make displaCy render straight from a dictionary
# # (if you're not running the code within a Jupyer environment, you can
# # use displacy.serve instead)
# displacy.render(matched_sents, style="ent", manual=True)

## PhraseMatching 

In [18]:

# nlp = spacy.load("en_core_web_sm")
# matcher = PhraseMatcher(nlp.vocab)
# terms = ["Barack Obama", "Angela Merkel", "Washington, D.C."]
# # Only run nlp.make_doc to speed things up
# patterns = [nlp.make_doc(text) for text in terms]
# matcher.add("TerminologyList", patterns)

# doc = nlp("German Chancellor Angela Merkel and US President Barack Obama "
#           "converse in the Oval Office inside the White House in Washington, D.C.")
# matches = matcher(doc)
# for match_id, start, end in matches:
#     span = doc[start:end]
#     print(span.text)

In [19]:
# from spacy.lang.en import English
# from spacy.matcher import PhraseMatcher

# nlp = English()
# matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
# patterns = [nlp.make_doc(name) for name in ["Angela Merkel", "Barack Obama"]]
# matcher.add("Names", patterns)

# doc = nlp("angela merkel and us president barack Obama")
# for match_id, start, end in matcher(doc):
#     print("Matched based on lowercase token text:", doc[start:end])

In [20]:
# matcher: Matcher = Matcher(dadma_nlp.vocab, validate=True)

# for t_index, text in enumerate(texts):
#     text = normalizer.normalize(text)
#     doc = stanza_nlp(text)
#     print(f'Text {t_index + 1}: ...')
        
#     for s_index, sentence in enumerate(doc.sents):
#         print(f'Sentence {s_index + 1}: ...')
#         for token in sentence:
#             print(f"{token.text:10}, {token.pos_:10}, {token.tag_:10}, {token.dep_}")
                
        
#     print("\n\n")

In [21]:
# from spacy import displacy

# for index, text in enumerate(texts):
#     text = normalizer.normalize(text)
#     try:
        
#         doc = dadma_nlp(text)
        
#         # print(f'sentence {index + 1}: ...')
#         # for token in doc:
#         #     print(f'word: {token.text:12}, pos: {token.pos_:10}, tag: {token.tag_:10}, dep: {token.dep_:15}')
#         #     print("\n")
        
#         sentences = doc._.sentences
        
#         # for sentence in sentences:
#         #     sentence_text = sentence.text
#         #     for token in sentence:
#         #         token_text = token.text
#         #         lemma = token.lemma_ ## this has value only if lem is called
#         #         pos_tag = token.pos_ ## this has value only if pos is called
#         #         dep = token.dep_ ## this has value only if dep is called
#         #         dep_arc = token._.dep_arc ## this has value only if dep is called
#         #         print(token_text, pos_tag, dep, dep_arc)
#         #         if token.pos_ == "AUX":
#         #             token.pos_ = "VERB"
                
#         sent_constituency = doc._.constituency ## this has value only if cons is called
#         sent_chunks = doc._.chunks ## this has value only if cons is called
#         # ners = doc._.ners ## this has value only if ner is called
#         # print(sent_constituency)
#         print(sent_chunks)
        
#         print("\n\n\n")
        
#         displacy.render(doc, style="dep")
        
#     except Exception:
        
#         print(f"ERRR {text}")
    
        

## [Pattern Matching](https://spacy.io/usage/spacy-101#architecture-matchers)

